In [ ]:
# ===============================================================
# STRATEGIC & TACTICAL SOLUTION: Predicting Customer Churn in E-Commerce Using ANN
# ===============================================================
# Project Context:
# This project demonstrates a strategic AI solution for predicting customer churn
# using an Artificial Neural Network (ANN). Churn prediction is critical for business
# strategy as it allows companies to proactively retain valuable customers.
#
# Tactical Relevance:
# - ANN can capture complex non-linear relationships between customer behavior,
#   purchase history, and engagement metrics.
# - Enables actionable insights by predicting the probability of churn.
#
# Dataset:
# Kaggle: "E-Commerce Customer Churn Analysis and Prediction"
# URL (for reference in read-only manner): Please refer to README.md file in Github
# ===============================================================

import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping

from IPython.display import display

# -----------------------------------
# STEP 1: Load Dataset (Read-Only)
# -----------------------------------
file_path = '/content/ANN/E-Commerce-Dataset.csv'
df = pd.read_csv(file_path)
print(f"E-Commerce Dataset loaded from {file_path} (read-only, Kaggle URL in README.md file in Github)")
print(df.head())

# -----------------------------------
# STEP 2: Data Inspection & Cleaning
# -----------------------------------
print("Dataset Shape:", df.shape)
print("\nColumn Names:", df.columns.tolist())
print("\nData Types:\n", df.dtypes)
df.info()
display(df.describe().T)
print("\nMissing Values:\n", df.isnull().sum())

for col in ['Tenure','WarehouseToHome','HourSpendOnApp','OrderAmountHikeFromlastYear',
            'CouponUsed','OrderCount','DaySinceLastOrder']:
    df[col] = df[col].fillna(0)

df = df.drop('CustomerID', axis=1)

# -----------------------------------
# STEP 3: Separate Numerical & Categorical Columns
# -----------------------------------
numerical_cols = df.select_dtypes(include=['int64','float64']).columns
categorical_cols = df.select_dtypes(include=['object','category']).columns

# Encode categorical columns & store mapping for unseen data
category_maps = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    category_maps[col] = dict(zip(le.classes_, le.transform(le.classes_)))

# -----------------------------------
# STEP 4: Exploratory Data Analysis (EDA)
# -----------------------------------
df['Churn'].value_counts().plot(kind='bar', color=['skyblue','salmon'])
plt.title("Churn Class Distribution")
plt.ylabel("Count")
plt.show()

important_numeric = ['Tenure','HourSpendOnApp','OrderCount','CashbackAmount']
df[important_numeric].hist(bins=20, figsize=(10,6), color='lightgreen')
plt.suptitle("Key Numeric Feature Distributions", fontsize=14)
plt.show()

plt.figure(figsize=(14,12))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

# -----------------------------------
# STEP 5: Handle Class Imbalance
# -----------------------------------
majority_class = df[df['Churn']==0]
minority_class = df[df['Churn']==1]

minority_over = resample(minority_class, replace=True, n_samples=len(majority_class), random_state=42)
df = pd.concat([majority_class, minority_over]).sample(frac=1, random_state=42).reset_index(drop=True)

print("\nBalanced class distribution:\n", df['Churn'].value_counts())

# -----------------------------------
# STEP 6: Train/Test Split & Feature Scaling
# -----------------------------------
X = df.drop('Churn', axis=1)
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# -----------------------------------
# STEP 7: Build ANN Model (Strategic & Tactical Explanation)
# -----------------------------------
# - Dense: Fully connected layer linking all neurons from previous layer
# - ReLU: Activation function introducing non-linearity, helping the model learn complex patterns
# - Dropout: Regularization technique; randomly disables neurons to prevent overfitting
# - Sigmoid: Output layer activation for binary classification (0=No churn, 1=Churn)
# - Adam optimizer: Adaptive learning rate optimizer for faster convergence

model = Sequential()

# Input layer (number of features = input shape)
model.add(Input(shape=(X_train.shape[1],)))

# Hidden Layer 1
model.add(Dense(128, activation='relu'))  # 128 neurons, ReLU activation
model.add(Dropout(0.1))  # Dropout 10% for regularization

# Hidden Layer 2
model.add(Dense(64, activation='relu'))  # 64 neurons, ReLU activation
model.add(Dropout(0.2))  # Dropout 20% for regularization

# Output Layer
model.add(Dense(1, activation='sigmoid'))  # 1 neuron, sigmoid activation for binary output

print("\n================ MODEL SUMMARY =================")
model.summary()
print("==============================================\n")

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# EarlyStopping: stops training if validation loss does not improve for 5 consecutive epochs
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# -----------------------------------
# STEP 8: Train ANN Model
# -----------------------------------
history = model.fit(X_train, y_train,
                    validation_split=0.2,
                    batch_size=64,
                    epochs=50,
                    callbacks=[early_stop])

# -----------------------------------
# STEP 9: Model Evaluation
# -----------------------------------
plt.figure(figsize=(8,5))
plt.plot(history.history['accuracy'], label='Training Accuracy', color='green', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='orange', linewidth=2)
plt.title("Training vs Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(8,5))
plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
plt.title("Training vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

y_pred = (model.predict(X_test) > 0.5).astype(int)

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

# -----------------------------------
# STEP 10: Predict on Unseen Data (Simple Display)
# -----------------------------------
# Purpose:
# Demonstrates how the trained ANN predicts churn for unseen customers.
# Churn_Probability: Likelihood of churn (0 to 1)
# Predicted_Label: 1 if probability > 0.5, else 0

unseen_data = pd.DataFrame([
    [4, 'Mobile Phone', 2, 6, 'Debit Card', 'Female', 2, 3, 'Laptop & Accessory', 5,
     'Single', 9, 1, 11, 1, 1, 5, 230],
    [0, 'Mobile Phone', 1, 6, 'Credit Card', 'Male', 1, 4, 'Laptop & Accessory', 3,
     'Divorced', 2, 0, 17, 1, 1, 0, 120]
], columns=X.columns)

# Encode categorical columns
for col in categorical_cols:
    unseen_data[col] = unseen_data[col].map(category_maps[col]).fillna(-1)

# Scale numeric features
unseen_scaled = scaler.transform(unseen_data)

# Predict probabilities
pred_probs = model.predict(unseen_scaled)
pred_labels = (pred_probs > 0.5).astype(int)

# Display results
results = unseen_data.copy()
results['Churn_Probability'] = pred_probs
results['Predicted_Label'] = pred_labels
results.index = [f'Customer_{i+1}' for i in range(len(results))]

print("\n=== PREDICTION RESULTS FOR UNSEEN CUSTOMERS ===\n")
display(results)

print("\nExplanation:")
print("- Churn_Probability: Higher values indicate higher likelihood of customer churn.")
print("- Predicted_Label: 1 = Likely to churn, 0 = Not likely to churn.")
